In [42]:
from google.colab import drive
drive.mount('/content/drive')

!cp drive/My\ Drive/bigdata/similarity/true-classes.npz drive/My\ Drive/bigdata/similarity/resnet50-pretrained-np-dump.npz drive/My\ Drive/bigdata/similarity/resnet50-qs-1282015-dump.npz.zip .
!unzip resnet50-qs-1282015-dump.npz.zip

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  resnet50-qs-1282015-dump.npz.zip
  inflating: resnet50-qs-1282015-dump.npz  


In [43]:
!ls

datasets	     resnet50-pretrained-np-dump.npz
drive		     resnet50-qs-1282015-dump.npz
extracted.txt	     resnet50-qs-1282015-dump.npz.zip
feature-dump.npz     resnet50-qs-dump.npz
nn-qs-answers.npz    sample_data
nn-true-answers.npz  true-classes.npz


In [0]:
#
# This code is for doing similarity search on set of images.
# Image features generated using:
# resnet pretrained-embeddings, resnet importance sampling trained
# Sketches (compressed representations) of features generated using
# grid, quadsketch, quadsketch variant
#

## imports
# libraries
import numpy as np
from numpy import linalg as LA

## gloabls
true_training_categories   = []
true_query_categories      = []
true_training_features     = []
true_query_features        = []
sketched_training_features = []
sketched_query_features    = []

true_answers = []
sketched_answers = []

DIM       = 2048
## functions

def query(Q, K):
    '''
    Q could be the original or the sketched feature vector
    Returns K nearest images
    '''
    pass

def evaluate(K):
    ## get class of vector

    ## original feature based nearest neighbor
    #print(len(true_query_features))
    #print(len(true_query_categories))
    #print(true_query_categories[0])
    ## for every query compute the NN
    
    print("# of queries:", len(true_query_features))
    
    print("Computing for true features...")
    percent = 0
    
    for q in range(len(true_query_features)):
        feature = true_query_features[q]
        category = true_query_categories[q]

        min_distance = 100000000
        who = -1     # is the nearest neighbor
        which = -1   # category
        if((q + 1) % 100 == 0):
            print(q + 1, " queries done...")
            percent += 10
        for r in range(len(true_training_features)):
            euc_dis = LA.norm(true_training_features[r] - true_query_features[q])
            if(euc_dis < min_distance):
                min_distance = euc_dis
                who = r # index in the database
                which = true_training_categories[r]
        #print(min_distance)
        true_answers.append([category, which, who])

    ## sketched feature based nearest neighbor
    print("\nComputing for sketched features...")
    percent = 0
    for q in range(len(sketched_query_features)):
        feature = sketched_query_features[q]
        category = true_query_categories[q]

        min_distance = 100000000
        who = -1     # is the nearest neighbor
        which = -1   # category
        if((q + 1) % 100 == 0):
            print(q + 1, "% queries done...")
            percent += 10
        for r in range(len(sketched_training_features)):
            euc_dis = LA.norm(sketched_training_features[r] - sketched_query_features[q])
            if(euc_dis < min_distance):
                min_distance = euc_dis
                who = r # index in the database
                which = true_training_categories[r]
        #print(min_distance)
        sketched_answers.append([category, which, who])
        
        
    # Computing distortion
    
    exact_counter = 0
    class_counter = 0
    distortion = 0.0
    for q in range(len(sketched_query_features)):
        true_answer_feature = true_training_features[true_answers[q][2]]
        dd2 = LA.norm(true_query_features[q] - true_answer_feature)
        
        if dd2 < 1e-3:
          distortion += 1.0
          exact_counter += 1
        else:
          dd1 = LA.norm(true_training_features[sketched_answers[q][2]] - true_query_features[q])
          distortion += dd1/dd2
          if(true_answers[q][2] == sketched_answers[q][2]): # exact nearest neighbor pt
            exact_counter += 1
          #else:
            # not the exact same neighbor, but same class?
            #if(true_answers[q][1] == sketched_answers[q][1]) # means qs, and original features predicted same class
              #class_counter += 1
              #if(true_answers[q][1] == true_answers[q][0])   # both predicted the correct category
            
                     
    print("EVAL")
    print(len(true_answers))
    print(len(sketched_answers))
    print("EVAL")
    
    ##
    distortion = distortion / len(sketched_query_features)
    accuracy = exact_counter / len(sketched_query_features)
    print("Avg Distortion: ", distortion)
    print("Accuracy:", accuracy)
    ##
    
    # save the answers?
    np.savez("nn-true-answers.npz", true_answers=true_answers)
    np.savez("nn-qs-answers.npz"  , sketched_answers=sketched_answers)
    np.savez("stats.npz", accuracy=accuracy, distortion=distortion)
    return true_answers, sketched_answers

In [98]:
import time
if __name__ == "__main__":

    ## decompressed
    # dataset =
    # queries =
    ##
    
    # first entry of both is [], for class indexing 1-257
    true_file = np.load("true-classes.npz", allow_pickle=True)
    train_ex = true_file['tr_paths'] # has first element as []
    query_ex = true_file['qr_paths'] # has first element as []
    for category in range(1, 258):
        for im in train_ex[category]:
            true_training_categories.append(int(im[1]))
        for im in query_ex[category]:
            true_query_categories.append(int(im[1]))

    ## from the colab file, as is, no change required
    true_features_file = np.load("resnet50-pretrained-np-dump.npz", allow_pickle=True)
    true_training_features = true_features_file['emb_training']
    true_query_features = true_features_file['emb_query']

    ## from the cpp codes, requires to segregate training and query
    sketched_features_file = np.load("resnet50-qs-1282015-dump.npz", allow_pickle=True)
    #print(sketched_features_file.files)
    all_sketched_features = sketched_features_file['qstq']
    all_sketched_features_N = [all_sketched_features[i * DIM: (i + 1) * DIM] for i in range(int(len(all_sketched_features)/2048))]
    NUM_QUERY  = len(true_query_features)
    NUM_TRAIN  = len(true_training_features)
    sketched_training_features = all_sketched_features_N[: NUM_TRAIN]
    sketched_query_features = all_sketched_features_N[NUM_TRAIN: len(all_sketched_features_N)]

    
    start = time.time()
    true_answers, sketched_answers = evaluate(1)
    end = time.time()
    print("Time reqired", end - start)

# of queries: 2963
Computing for true features...
100  queries done...
200  queries done...
300  queries done...
400  queries done...
500  queries done...
600  queries done...
700  queries done...
800  queries done...
900  queries done...
1000  queries done...
1100  queries done...
1200  queries done...
1300  queries done...
1400  queries done...
1500  queries done...
1600  queries done...
1700  queries done...
1800  queries done...
1900  queries done...
2000  queries done...
2100  queries done...
2200  queries done...
2300  queries done...
2400  queries done...
2500  queries done...
2600  queries done...
2700  queries done...
2800  queries done...
2900  queries done...

Computing for sketched features...
100 % queries done...
200 % queries done...
300 % queries done...
400 % queries done...
500 % queries done...
600 % queries done...
700 % queries done...
800 % queries done...
900 % queries done...
1000 % queries done...
1100 % queries done...
1200 % queries done...
1300 % queries don

In [0]:
!mkdir -p drive/My\ Drive/bigdata/similarity/results/
!cp nn-true-answers.npz drive/My\ Drive/bigdata/similarity/results/
!cp nn-qs-answers.npz drive/My\ Drive/bigdata/similarity/results/
!cp stats.npz drive/My\ Drive/bigdata/similarity/results/

In [76]:
TEST_QUERY = 10
print(true_answers[TEST_QUERY][0], true_answers[TEST_QUERY][1], true_answers[TEST_QUERY][2])  # true cat, computed cat, because of which vector
print(sketched_answers[TEST_QUERY][0], sketched_answers[TEST_QUERY][1], sketched_answers[TEST_QUERY][2])




2 135 13608
2 135 13608


In [41]:
from pprint import pprint
pprint(sketched_training_features[0])
pprint(sketched_training_features[1])
pprint(sketched_query_features[0])

array([0.5    , 0.53125, 0.5    , ..., 0.5    , 0.5    , 0.53125],
      dtype=float32)
array([0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5], dtype=float32)
array([0.5    , 0.53125, 0.5    , ..., 0.5    , 0.5    , 0.53125],
      dtype=float32)


In [100]:
!ls -l

total 863836
drwxr-xr-x 3 root root      4096 Apr 30 02:24 datasets
drwx------ 3 root root      4096 Apr 30 02:23 drive
-rw-r--r-- 1 root root   1484977 Apr 30 02:24 extracted.txt
-rw-r--r-- 1 root root    803304 Apr 30 02:29 feature-dump.npz
-rw-r--r-- 1 root root     71378 Apr 30 07:25 nn-qs-answers.npz
-rw-r--r-- 1 root root     71370 Apr 30 07:25 nn-true-answers.npz
-rw------- 1 root root 250733032 Apr 30 06:20 resnet50-pretrained-np-dump.npz
-rw-rw-r-- 1 root root 250732738 Apr 30 00:08 resnet50-qs-1282015-dump.npz
-rw------- 1 root root 127796646 Apr 30 06:20 resnet50-qs-1282015-dump.npz.zip
-rw------- 1 root root 250732738 Apr 30 04:45 resnet50-qs-dump.npz
drwxr-xr-x 1 root root      4096 Apr 26 16:21 sample_data
-rw-r--r-- 1 root root       498 Apr 30 07:25 stats.npz
-rw-r--r-- 1 root root   2092181 Apr 30 06:20 true-classes.npz
